## Lab 7 - Forwards and Futures - Part 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# This is an example of how to use Enum type in Python

from enum import Enum
 
class Season(Enum):
    SPRING = 1
    SUMMER = 2
    AUTUMN = 3
    WINTER = 4
 
# printing enum member as string
print(Season.SPRING)
 
# printing name of enum member using "name" keyword
print(Season.SPRING.name)
 
# printing value of enum member using "value" keyword
print(Season.SPRING.value)
 
# printing the type of enum member using type()
print(type(Season.SPRING))
 
# printing enum member as repr
print(repr(Season.SPRING))
 
# printing all enum member using "list" keyword
print(list(Season))

Season.SPRING
SPRING
1
<enum 'Season'>
<Season.SPRING: 1>
[<Season.SPRING: 1>, <Season.SUMMER: 2>, <Season.AUTUMN: 3>, <Season.WINTER: 4>]


In [4]:
from enum import Enum
 
class AssetType(Enum):
    NO_INCOME = 1
    KNOWN_INCOME = 2
    KNOWN_CONTINUOUS_YIELD = 3

# Exercise 1 (2 pts) - Forward price of an asset

Implement a function to return the forward price of an asset that could be one of the three following asset types
* `AssetType.NO_INCOME`: pays no coupons or dividends. 
* `AssetType.KNOWN_INCOME`: pays coupons or discrete cashflows
* `AssetType.KNOWN_CONTINUOUS_YIELD`: pays dividend yield
Use continuous compounding.
I (income) is given as a tuple of (cashflow, time). For example, the income of a bond that pays four 5-dollar coupons in 6 months, 1 year, 1.5 years, and 2 years can be represented as $((5, 0.5), (5, 1), (5, 1.5), (5, 2))$
The function should check the `asset_type` of the asset to use the correct formula for $F_t$.

In [5]:
def calcForwardPrice(S, r, t, T, I, q, asset_type):
    
    """
    S: Spot price of the asset at time t
    r: risk-free interest rate
    t: current time
    T: maturity (in years)
    I: Present value at time t of the income received in [t, T] 
    q: dividend yield
    asset_type: AssetType
    
    
    return:
    F: Forward price calculated at time t (Ft)
    """
    
    # Your code goes here
    if asset_type == AssetType.NO_INCOME:
        Ft = S * np.exp(r * (T - t))
    elif asset_type == AssetType.KNOWN_INCOME:
        Ft = (S - I) * np.exp(r * (T - t))
    elif asset_type == AssetType.KNOWN_CONTINUOUS_YIELD:
        Ft = S * np.exp((r - q) * (T - t))
    else:
        raise ValueError("Invalid asset type")
    
    return Ft
    

# Exercise 2 (2 pts) - Valuing a forward contract

Implement a function to return the value at time t of a forward contract. Use continuous compounding. Use function `calcForwardPrice()` to calculate $F_t$

In [6]:
def calForwardValue(S, r, K, t, T, I, q, asset_type):
    
    """
    S: Spot price of the asset at time t
    r: risk-free interest rate
    t: current time
    T: maturity (in years)
    I: Present value at time t of the income received in [t, T] 
    q: dividend yield
    asset_type: AssetType
    
    
    return:
    v: Forward price calculated at time t (Ft)
    """
    
    # Your code goes here
    Ft = calcForwardPrice(S, r, t, T, I, q, asset_type)
    
    v = (Ft - K) * np.exp(-r * (T - t))
    return v

# Exercise 3 (1 points)

Implement a function to calculate the forward rate at time $t=0$ for the period $[t_1, t_2]$. 

In [7]:
def calForwardRate0(t1, t2, r1, r2):
    
    """
    r1: interest rate for period [0, t1]
    r2: interest rate for period [0, t2]
    
    return: r, forward rate for period [t1, t2]
    """
    
    # Your code goes here
    r = (r2*t2 - r1*t1) / (t2 - t1)
    
    return r

# Exercise 4 (1 points)

Implement a function to calculate the forward rate at time $t, \ 0 \leq t \leq t_1$ for the period $[t_1, t_2]$. 

In [9]:
def calForwardRate(t, t1, t2, r1, r2):
    
    """
    t: the time at which the forward rate is calculated
    r1: interest rate for period [0, t1]
    r2: interest rate for period [0, t2]
    
    
    return: r, forward rate calculated at time t for the period [t1, t2]
    """
    
    # Your code goes here
    df1 = 1 / (1 + r1 * (t1 - t))
    df2 = 1 / (1 + r2 * (t2 - t))

    r = df1 / df2 - 1
    
    return r

# Exercise 5 (2 pts) - Valuing FRA

An FRA was entered at time $0$ for the period $[t_1, t_2]$. Write a function to evaluate the FRA to the lender at time $t, \ 0 \leq t \leq t_1$. 

$V_t=L e^{-r_2^*\left(t_2-t\right)}\left(e^{r_K\left(t_2-t_1\right)}-e^{r_t^f\left(t_2-t_1\right)}\right)$

You can use the functions `calForwardRate0()` and `calForwardRate()` written above to calculate 

* $r_K$, the forward rate for $[t_1, t_2]$ as seen at time $0$.
* $r_t^f$, the forward rate for $[t_1, t_2]$ as seen at time $t$.

Use your function to solve the following problem.

GM needs to borrow $1 million in 2 months. The money will be borrowed for 8 months. 2- and 10-month interest rates are 5.24\% and 5.35\%, respectively. GM enters an FRA with a bank. Suppose in 2 months, the 8-month LIBOR rate turns out to be 5.6\%. How should the FRA be settled?

In [12]:
def calFraValue(t, t1, t2, r1, r2, r1s, r2s):
    
    """
    t: the time at which the forward rate is valued
    r1: interest rate for period [0, t1]
    r2: interest rate for period [0, t2]
    r1s: interest rate for period [t, t1]
    r2s: interest rate for period [t, t2]
    rtf: forward rate calculated at time t for the period [t1, t2]
    
    return: v, value of the FRA at time t
    """
    
    # Your code goes here
    rtf = calForwardRate(t, t1, t2, r1, r2)
    
    df1s = 1 / (1 + r1s * (t1 - t))
    df2s = 1 / (1 + r2s * (t2 - t))

    v = (rtf - r2s) * (t2 - t1) * df1s * df2s
    
    return v

In [15]:
# GM needs to borrow $1 million in 2 months. The money will be borrowed for 8 months. 
# 2- and 10-month interest rates are 5.24% and 5.35%, respectively. 
# GM enters an FRA with a bank. Suppose in 2 months, the 8-month LIBOR rate turns out 
# to be 5.6%. How should the FRA be settled?

# Your code goes here
r = calForwardRate0(2/12, 10/12, 0.0524, 0.0535)
v = 1000000 * (1 - np.exp((r - 0.056)*8/12))
print(v)
 

1482.2337382003336


# Exercise 6 (1 pt) - Forward price of a commodity as an investment asset

The forward price of a commodity as an investment asset is given by 

$$F_0=\left(S_0+U\right) e^{r T}$$

where 

$F_0$: forward price of a unit of the commodity calculated at time $0$\
$S_0$: spot proce of a unit of the commodity at time $0$\
$U$: the present value of storage costs for each unit of the asset\
$r$: risk-free interest rate\
$T$: maturity (in years)

Implement a function to return the value at time t of a forward contract. Use continuous compounding.

In [16]:
def calcForwardPriceCommodities(S, U, r, T):
    
    """
    S: Spot price of a unit of the commodity at time 0
    𝑈: present value of storage costs for each unit of the commodity
    r: risk-free interest rate
    T: maturity (in years)
    
    return:
    F: Forward price of a unit of the commodity calculated at time 0 (F0)
    """
    
    # Your code goes here
    F = (S + U) * np.exp(r * T)
    
    return F

# Exercise 7 (1 pt) - Convenience yield of a consumption commodity

The convenience yield $y$ of a consumption commodity satistifies 
$$F_0 e^{y T}=\left(S_0+U\right) e^{r T}$$

where 

$F_0$: forward price of a unit of the commodity at time $0$\
$S_0$: spot price of a unit of the commodity at time $0$\
$U$: the present value of storage costs for each unit of the asset\
$r$: risk-free interest rate\
$T$: maturity (in years)

Implement a function to return the convenience yield of a forward contract. Use continuous compounding.

In [17]:
def calcConvenienceYield(S, U, F, r, T):
    
    """
    S: Spot price of a unit of the commodity at time 0
    𝑈: present value of storage costs for each unit of the asset
    r: risk-free interest rate
    T: maturity (in years)
    F: Forward price of a unit of the commodity at time 0 (F0)
    
    return:
    y: convenience yield
    """
    
    # Your code goes here
    y = np.log((F - U) / S) / T - r
    
    return y